<a href="https://colab.research.google.com/github/qin-na/SZPT-Q/blob/main/polymerNER%E5%BE%AE%E8%B0%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install datasets transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d56b68f3afcea06f55f68c8bf68e373222e4d9ad0da6ede51fcb9a8a2d3c4d0d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5

In [4]:
!pip install transformers==4.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0
    Uninstalling transformers-4.32.0:
      Successfully uninstalled transformers-4.32.0


In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "pranav-s/PolymerNER"
batch_size = 16
truncation=True
# 加载数据
from datasets import load_dataset, load_metric, load_from_disk
datasets = load_from_disk("/content/drive/MyDrive/autotrain-data-automatic/processed")
#datasets
datasets["train"][0]
datasets["train"].features[f"tags"]
label_list = datasets["train"].features[f"tags"].feature.names
#label_list
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
#show_random_elements(datasets["train"])

# 预处理数据
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
tokenizer("Hello, this is one sentence!")
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)
example = datasets["train"][4]
print(example["tokens"])
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
len(example[f"tags"]), len(tokenized_input["input_ids"])
print(tokenized_input.word_ids())
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], max_length=512, truncation=True,return_tensors='pt', padding=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenize_and_align_labels(datasets['train'][:5])
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
# 微调模型
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list),ignore_mismatched_sizes=True,max_length=512)
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    weight_decay=0.01,
    #push_to_hub=True,
    push_to_hub=False
)
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"tags"]]
metric.compute(predictions=[labels], references=[labels])
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

['Exploring', 'hydrogen', 'evolution', 'reaction', '(HER)', 'catalyst', 'with', 'highly', 'catalytic', 'features', 'in', 'alkaline', 'conditions', 'is', 'considered', 'as', 'significance', 'for', 'water', 'splitting.', 'In', 'this', 'study,', 'a', 'general', 'and', 'simple', 'method', 'was', 'developed', 'to', 'prepare', 'flower-like', 'platinum-cobalt-ruthenium', 'alloy', 'nanoassemblies', '(PtCoRu', 'NAs)', 'by', 'using', 'murexide', 'and', 'cetyltrimethylammonium', 'chloride', '(CTAC)', 'as', 'the', 'co-structure-directing', 'agents.', 'Benefiting', 'from', 'the', 'structural', 'advantages', 'and', 'multimetallic', 'compositions,', 'the', 'as-prepared', 'PtCoRu', 'NAs', 'displayed', 'remarkably', 'enhanced', 'electrocatalytic', 'performance', 'for', 'the', 'HER', 'in', '1.0', 'M', 'KOH,', 'with', 'a', 'low', 'overpotential', '(η,', '22', 'mV', ')', 'to', 'drive', '10', 'mA', 'cm<sup>-2</sup>', ',', 'small', 'Tafel', 'slope', '(', '46', 'mV', 'dec<sup>-1</sup>', '),', 'and', 'high', 

Some weights of BertForTokenClassification were not initialized from the model checkpoint at pranav-s/PolymerNER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-16dfe6ca56c7>:102: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CATALYST seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPERTY_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPERTY_VALUE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a fu

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.351813,0.462921,0.438298,0.450273,0.896381
2,No log,0.215172,0.596899,0.655319,0.624746,0.929811
3,No log,0.204281,0.638095,0.712766,0.673367,0.933069
4,No log,0.189528,0.687885,0.712766,0.700104,0.939231
5,No log,0.188789,0.755556,0.723404,0.739130,0.943339
6,No log,0.187789,0.753846,0.729787,0.741622,0.944260
7,No log,0.203114,0.734151,0.763830,0.748697,0.944401
8,No log,0.213683,0.746392,0.770213,0.758115,0.944047
9,No log,0.216680,0.742394,0.778723,0.760125,0.941781
10,No log,0.247724,0.722656,0.787234,0.753564,0.940010


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 42
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CO-CATALYST seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags ar

TrainOutput(global_step=1100, training_loss=0.036563156788999386, metrics={'train_runtime': 1789.4141, 'train_samples_per_second': 9.389, 'train_steps_per_second': 0.615, 'total_flos': 4389944264294400.0, 'train_loss': 0.036563156788999386, 'epoch': 100.0})

In [2]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 42
  Batch size = 16


{'eval_loss': 0.38741952180862427,
 'eval_precision': 0.7455268389662028,
 'eval_recall': 0.7978723404255319,
 'eval_f1': 0.7708119218910585,
 'eval_accuracy': 0.9441178553721935,
 'eval_runtime': 1.7517,
 'eval_samples_per_second': 23.976,
 'eval_steps_per_second': 1.713,
 'epoch': 100.0}

In [3]:
predictions, labels, _ = trainer.predict(tokenized_datasets["valid"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 42
  Batch size = 16


{'ATALYST': {'precision': 0.6157635467980296,
  'recall': 0.7062146892655368,
  'f1': 0.6578947368421053,
  'number': 177},
 'CATALYST': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'ROPERTY_NAME': {'precision': 0.8142857142857143,
  'recall': 0.8636363636363636,
  'f1': 0.8382352941176471,
  'number': 132},
 'ROPERTY_VALUE': {'precision': 0.85,
  'recall': 0.8831168831168831,
  'f1': 0.8662420382165604,
  'number': 154},
 'overall_precision': 0.7455268389662028,
 'overall_recall': 0.7978723404255319,
 'overall_f1': 0.7708119218910585,
 'overall_accuracy': 0.9441178553721935}